#### What is the effect of product layout on purchasing behaviour in sales ?

In [2]:
import pandas as pd

df = pd.read_excel('../QVI_results.xlsx')

df.info()
df.drop(columns=['MOIS'], inplace=True)



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 264836 entries, 0 to 264835
Data columns (total 14 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   DATE              264836 non-null  datetime64[ns]
 1   MOIS              1564 non-null    float64       
 2   STORE_NBR         264836 non-null  int64         
 3   LYLTY_CARD_NBR    264836 non-null  int64         
 4   LIFESTAGE         264836 non-null  object        
 5   PREMIUM_CUSTOMER  264836 non-null  object        
 6   TXN_ID            264836 non-null  int64         
 7   PROD_NBR          264836 non-null  int64         
 8   PROD_NAME         264836 non-null  object        
 9   BRAND             264836 non-null  object        
 10  SIZE              264836 non-null  object        
 11  PRODUCT           264836 non-null  object        
 12  PROD_QTY          264836 non-null  int64         
 13  TOT_SALES         264836 non-null  float64       
dtypes: d

In [3]:
# Trial stores dataframe
trial_store = df[df['STORE_NBR'].isin([77, 86, 88])][['DATE', 'STORE_NBR', 'LYLTY_CARD_NBR', 'TOT_SALES']]
trial_store.head()

,DATE,STORE_NBR,LYLTY_CARD_NBR,TOT_SALES
69,2019-05-20,88,88320,9.2
202,2019-06-17,77,77069,8.4
1438,2019-03-28,77,77000,3.3
1439,2019-04-13,77,77000,3.0
1440,2018-07-17,77,2330211,2.3


In [4]:
# Group by store and month, then calculate metrics
monthly_metrics = trial_store.groupby([
    'STORE_NBR', 
    pd.Grouper(key='DATE', freq='M')
]).agg({
    'TOT_SALES': 'sum',  # Monthly sales revenue
    'LYLTY_CARD_NBR': ['nunique', 'count']  # Unique customers and total transactions
}).reset_index()

# Rename columns for clarity
monthly_metrics.columns = [
    'STORE_NBR', 
    'MONTH', 
    'TOTAL_SALES', 
    'NUM_CUSTOMERS',
    'NUM_TRANSACTIONS'
]

# Calculate transactions per customer
monthly_metrics['TRANS_PER_CUSTOMER'] = monthly_metrics['NUM_TRANSACTIONS'] / monthly_metrics['NUM_CUSTOMERS']

# Sort by store and month
monthly_metrics = monthly_metrics.sort_values(['STORE_NBR', 'MONTH'])

print("Monthly metrics by store:")
print(monthly_metrics)


Monthly metrics by store:
    STORE_NBR      MONTH  TOTAL_SALES  NUM_CUSTOMERS  NUM_TRANSACTIONS  \
0          77 2018-07-31       296.80             51                55   
1          77 2018-08-31       255.50             47                48   
2          77 2018-09-30       225.20             42                44   
3          77 2018-10-31       204.50             37                38   
4          77 2018-11-30       245.30             41                44   
5          77 2018-12-31       267.30             46                49   
6          77 2019-01-31       204.40             35                39   
7          77 2019-02-28       235.00             45                45   
8          77 2019-03-31       278.50             50                55   
9          77 2019-04-30       263.50             47                48   
10         77 2019-05-31       299.30             55                56   
11         77 2019-06-30       264.70             41                42   
12         8

C:\Users\PC\AppData\Local\Temp\ipykernel_17720\1408615263.py:4: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  pd.Grouper(key='DATE', freq='M')


In [5]:
# Get control stores (all stores except trial stores)
control_stores = df[~df['STORE_NBR'].isin([77, 86, 88])][['DATE', 'STORE_NBR', 'LYLTY_CARD_NBR', 'TOT_SALES']]

# Group by store and month, calculate same metrics
control_monthly_metrics = control_stores.groupby([
    'STORE_NBR',
    pd.Grouper(key='DATE', freq='M')
]).agg({
    'TOT_SALES': 'sum',
    'LYLTY_CARD_NBR': ['nunique', 'count']
}).reset_index()

# Rename columns to match trial metrics
control_monthly_metrics.columns = [
    'STORE_NBR',
    'MONTH', 
    'TOTAL_SALES',
    'NUM_CUSTOMERS',
    'NUM_TRANSACTIONS'
]

# Calculate transactions per customer
control_monthly_metrics['TRANS_PER_CUSTOMER'] = control_monthly_metrics['NUM_TRANSACTIONS'] / control_monthly_metrics['NUM_CUSTOMERS']

# Get min and max dates from dataset
date_range = pd.date_range(start=df['DATE'].min(), end=df['DATE'].max(), freq='M')

# Filter for stores that have data for all months
store_counts = control_monthly_metrics.groupby('STORE_NBR').size()
complete_stores = store_counts[store_counts == len(date_range)].index

control_monthly_metrics = control_monthly_metrics[
    control_monthly_metrics['STORE_NBR'].isin(complete_stores)
].sort_values(['STORE_NBR', 'MONTH'])

print("\nMonthly metrics for control stores (with complete data):")
print(control_monthly_metrics)



Monthly metrics for control stores (with complete data):
      STORE_NBR      MONTH  TOTAL_SALES  NUM_CUSTOMERS  NUM_TRANSACTIONS  \
0             1 2018-07-31        206.9             49                52   
1             1 2018-08-31        176.1             42                43   
2             1 2018-09-30        278.8             59                62   
3             1 2018-10-31        188.1             44                45   
4             1 2018-11-30        192.6             46                47   
...         ...        ...          ...            ...               ...   
3128        272 2019-02-28        395.5             45                48   
3129        272 2019-03-31        442.3             50                53   
3130        272 2019-04-30        445.1             54                56   
3131        272 2019-05-31        314.6             34                40   
3132        272 2019-06-30        312.1             34                37   

      TRANS_PER_CUSTOMER  
0 

C:\Users\PC\AppData\Local\Temp\ipykernel_17720\3862752838.py:7: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  pd.Grouper(key='DATE', freq='M')
C:\Users\PC\AppData\Local\Temp\ipykernel_17720\3862752838.py:26: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  date_range = pd.date_range(start=df['DATE'].min(), end=df['DATE'].max(), freq='M')


In [6]:
control_stores

,DATE,STORE_NBR,LYLTY_CARD_NBR,TOT_SALES
0,2018-10-17,1,1000,6.0
1,2019-05-14,1,1307,6.3
2,2019-05-20,1,1343,2.9
3,2018-08-17,2,2373,15.0
4,2018-08-18,2,2426,13.8
